## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-22-02-23-15 +0000,nypost,Trump’s special counsel pick Paul Ingrassia wi...,https://nypost.com/2025/10/21/us-news/trumps-s...
1,2025-10-22-02-04-22 +0000,nyt,As Trump Moves Forward With White House Ballro...,https://www.nytimes.com/2025/10/21/arts/design...
2,2025-10-22-02-03-28 +0000,48hills,The surreal world of the supes and the Lurie [...,https://48hills.org/2025/10/the-surreal-world-...
3,2025-10-22-02-02-05 +0000,bbc,Australian PM defends US ambassador after Trum...,https://www.bbc.com/news/articles/cpv14jlm7dyo...
4,2025-10-22-02-00-00 +0000,wsj,Mamdani Is Promising a Cheaper New York. But C...,https://www.wsj.com/politics/elections/zohran-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2324/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,52
16,house,17
67,up,15
15,white,14
259,gaza,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
158,2025-10-21-18-44-04 +0000,bbc,From bowling alley to ballroom - Trump isn't t...,https://www.bbc.com/news/articles/cr7m2v80dy4o...,109
265,2025-10-21-11-43-48 +0000,cbc,White House demolition work begins as Trump pu...,https://www.cbc.ca/news/world/us-white-house-d...,108
170,2025-10-21-17-55-57 +0000,nypost,Hillary mocked after blasting Trump’s White Ho...,https://nypost.com/2025/10/21/us-news/hillary-...,106
345,2025-10-21-02-48-43 +0000,bbc,White House begins demolishing part of East Wi...,https://www.bbc.com/news/articles/ced6np51532o...,104
1,2025-10-22-02-04-22 +0000,nyt,As Trump Moves Forward With White House Ballro...,https://www.nytimes.com/2025/10/21/arts/design...,100


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
158,109,2025-10-21-18-44-04 +0000,bbc,From bowling alley to ballroom - Trump isn't t...,https://www.bbc.com/news/articles/cr7m2v80dy4o...
71,74,2025-10-21-22-12-52 +0000,nyt,Vance Lands in Israel as U.S. Tries to Shore U...,https://www.nytimes.com/2025/10/21/world/middl...
39,53,2025-10-21-23-38-00 +0000,wsj,Paul Ingrassia withdrew Tuesday from a Senate ...,https://www.wsj.com/politics/policy/paul-ingra...
247,43,2025-10-21-13-25-26 +0000,nypost,Former French President Sarkozy begins 5-year ...,https://nypost.com/2025/10/21/world-news/forme...
200,42,2025-10-21-16-33-45 +0000,wapo,CNN’s future at stake as owner Warner Bros. Di...,https://www.washingtonpost.com/business/2025/1...
25,41,2025-10-22-00-24-39 +0000,nypost,Hunter Biden says watching Obama lead Joe off ...,https://nypost.com/2025/10/21/us-news/hunter-b...
246,40,2025-10-21-13-32-06 +0000,nyt,"Suspect Charged in Paris Museum Heist (No, Not...",https://www.nytimes.com/2025/10/21/world/europ...
324,40,2025-10-21-08-05-05 +0000,nyt,"Sanae Takaichi is Japan’s New Prime Minister, ...",https://www.nytimes.com/2025/10/21/world/asia/...
171,39,2025-10-21-17-44-24 +0000,nypost,Hamas released bodies of two more dead hostage...,https://nypost.com/2025/10/21/world-news/hamas...
55,34,2025-10-21-23-10-23 +0000,nypost,Netflix shares slide on rare earnings miss — s...,https://nypost.com/2025/10/21/media/netflix-sh...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
